In [1]:
import json
import itertools
from bokeh.plotting import figure, show, vplot, gridplot
from bokeh.io import output_notebook
from bokeh.models import LinearAxis, Range1d, BoxAnnotation, GridPlot
from pykalman import KalmanFilter
import numpy as np

In [2]:
class SimpleFilter(object):
    # extend filter
    def __init__(self):
        self.eest = 10  # error in estimate - given
        self.emea = 10  # error in measurement - given and doesn't change
        self.est = 10  # current estimate - given
        self.mea = -1  # current measurement - given
        
        self.kg = -1  # kalman gain - calculate
        self.prev_est = -1  # previous estimate - base case = est
        self.prev_eest = -1

    def __str__(self):
        return str(self.val)

    def __repr__(self):
        return str(self.val)

    def put(self, val):
        self.mea = val  # set current measurement
        
        # calculate kalman gain
        self.kg = self.eest/(self.eest + self.emea)
        
        # calculate current estimate
        self.prev_est = self.est
        self.est = self.prev_est + (self.kg*(self.mea - self.prev_est))
        
        # calculate the new estimate error
        self.prev_eest = self.eest
        self.eest = (1-self.kg) * (self.prev_eest)
        
    #def get(val):
    def get(self):
        return self.est

In [3]:
# global variables
initial_time = 0
times = []
filterD = []
filterR = []
begin = []
end = []
plots = []
original = []

In [4]:
def plot_minor(b):
    global times
    global filterD
    global filterR
    global begin
    global end
    global initial_time
    global original
    global plots
    tempTitle="Beacon 0:" + str(b)
    
    output_notebook()
    p = figure(
        tools="pan, box_zoom, wheel_zoom, reset, resize",
        title=tempTitle, x_axis_label='Time Elapsed (seconds)', y_axis_label='Distance (m)',
        x_range=((min(begin)-initial_time)/1000, (max(end)-initial_time)/1000), y_range=(0, max(original)),
        plot_width=900, plot_height=400
    )
    
    for i in range(len(begin)):
        box = BoxAnnotation(
            plot=p,
            left=(begin[i]-initial_time)/1000,
            right=(end[i]-initial_time)/1000,
            fill_alpha=0.1,
            fill_color='grey'
        )
        p.renderers.extend([box])
        
    p.scatter(
        times, original, marker="circle",
        fill_color="firebrick", line_color="firebrick",
        fill_alpha=0.4, legend="Unfiltered"
    )
    
    p.line(
        times, filterD, 
        line_color = "navy", line_width=3,
        legend="Filtered (Kalman)"
    )
    #p.line(times, original, line_color = "firebrick")
    
    '''
    p.extra_y_ranges = {"RSSI": Range1d(start=-100, end=0)}  # add y axis for rssi
    p.line(
        times, filterR,
        line_color = "firebrick", y_range_name="RSSI", legend="Filtered RSSI"
    )
    p.add_layout(LinearAxis(y_range_name="RSSI", axis_label="RSSI"), 'left')
    '''
     
    #plots.append(p)
    show(p)

In [5]:
def get_data(i):  # open text file and extract data
    global initial_time
    global times
    global filterD
    global filterR
    global original
    
    #textName = "minor" + str(i) + ".txt"
    textName = "2_6_2016_1441.txt"
    tempName = "0:" + str(i)
    f = open(textName)
    first = True
    pos = 0
    tempD = SimpleFilter()
    tempR = SimpleFilter()
    times[:] = []
    original[:] = []
    filterD[:] = []
    filterR[:] = []
    for line in f:
        words = line.split()
        time = int(words[0])  # get time
        if first == True:
            initial_time = time  # set base time
            first = False
        time = (time - initial_time) / 1000  # get elapsed time in seconds
        parsed_json = json.loads(words[1])  # parse json
        if parsed_json.get('node') and parsed_json['minor'] == i:
            beaconNum = str(parsed_json['major']) + ":" + str(parsed_json['minor'])  # beacon identified by major and minor
            rssi = parsed_json['rssi']
            distance = 10**((rssi+60)/-20)  # log-distance pathloss model
            tempD.put(distance)
            tempR.put(rssi)
            #val = kalman_filter(distance)
            filterD.append(tempD.get())
            filterR.append(tempR.get())
            times.append(time)
            original.append(distance)
    f.close()  # close file

In [6]:
def get_markers():
    global begin
    global end
    m = open("markers.txt")
    for line in m:
        words = line.split()
        time = int(words[0])  # get raw time
        parsed_json = json.loads(words[1])  # parse json
        if parsed_json['marker'] == "begin\n":
            begin.append(time)
        elif parsed_json['marker'] == "end\n":
            end.append(time)
    m.close()

In [7]:
def main():
    global plots
    global times
    
    plots.clear()
    
    get_markers()
    
    for i in range(7):
        get_data(i)
        plot_minor(i)
    
    #output_notebook()
    #grid = gridplot([plots])
    #grid = gridplot([plots[0:2], plots[2:4], plots[4:6], plots[6:8]])
    #show(grid)

In [8]:
if __name__ == "__main__":
    main()

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...